In [2]:
# coding: utf-8
import textwrap
import import_ipynb
import os
import Get_filelist as gf
import Read_pdf as rp
import Write_pdf as wp
import clipboard
import tkinter as tk
import pdfplumber
from shutil import copyfile
from tkinter import ttk
from tkinter import scrolledtext
from tkinter import messagebox
from tkinter import Menu
from tkinter import filedialog
from tkinter import simpledialog


class App:
    def __init__(self, master):
    #==============parameter set===============
        self.path = 'd:/'  #之後讓使用者定義預設資料夾

        
    #==============GUI set====================
        #----root and frame set
        
        self.master = master
        self.master.title('自動決包說明輔助工具')
#         self.master.geometry('760x210')
        
        self.lf = ttk.Frame(self.master)
        self.rf1 = ttk.Frame(self.master)
        self.rf2 = ttk.Frame(self.master)
        self.bf = ttk.Frame(self.master)
        
        self.lf.grid(column = 0 , row = 0, rowspan = 2, padx = 5, sticky = 'NWSE')
        self.rf1.grid(column = 1 , row = 0, padx = 5, sticky = 'NWSE')
        self.rf2.grid(column = 1 , row = 1, padx = 5, pady = 8, sticky = 'NWSE')
        self.bf.grid(column = 0 , row = 2, columnspan = 2 , sticky = 'NWSE')
    
    #=================Bottom frame widget=============================
        #----Status Label
        self.statusbar = tk.Label(self.bf, text = '狀態列', bd = 1, relief= tk.SUNKEN, anchor = tk.W)
        self.statusbar.pack(fill = tk.X)
    
    #==============left frame widget set=================
        #---Label
        self.lbl_pdfList = tk.Label(self.lf, text = 'PDF檔案:')
        self.lbl_pdfList.pack(side = tk.TOP, anchor = tk.NW)
        #----Treeview
        head = ('案件名稱','路徑')
        self.tree = ttk.Treeview(self.lf, show = 'tree', height = 6)
        self.tree["columns"]=head
        self.tree.column("#0", width=200, minwidth=20)
        self.tree.column(head[0], width=200, minwidth=20)
        self.tree.column(head[1], width=0, minwidth=0)
        
        self.tree_refresh()
        self.tree.bind('<ButtonRelease-1>', self.selectItem)
        self.tree.tag_configure('folder', background= "old lace")
        self.tree.pack()
        
        #----Button
        self.btn_refresh = ttk.Button(self.lf, text = '刪除PDF', state =tk.DISABLED, command = self.remove)
        self.btn_refresh.pack(anchor = tk.SE, pady = 8)
        
        
        
    #==============Right frame1 widget set=================
        #----Label
        self.lbl_text = tk.Label(self.rf1, text = '議價說明:')
        self.lbl_text.grid(row = 0, column = 0 , sticky = tk.W)
        
        #---OptionMenu
        self.opt_var = tk.StringVar(self.master)
        self.opt = ttk.OptionMenu(self.rf1, self.opt_var,\
                                  command = self.OptionMenu_SelectionEvent)
        
        self.opt.grid(row = 0 , column = 1 , sticky = tk.E)
        
        #----scrolledtext widget議價說明
        self.txt = tk.scrolledtext.ScrolledText(self.rf1, width = 58, height = 9)
        self.txt.grid(column = 0 , columnspan = 2 , row = 1)
        
    #==============Right frame2 widget set=================
        #----Buttons
        self.btn_printPdf = ttk.Button(self.rf2, text = '列印PDF', command = self.printPDF)
        self.btn_printPdf.pack(side = tk.RIGHT)
        
        self.btn_openPdf = ttk.Button(self.rf2, text = '打開PDF', command = self.openPDF)
        self.btn_openPdf.pack(side = tk.RIGHT, padx = 5)
        
        self.btn_writePdf = ttk.Button(self.rf2, text = '寫入PDF', command = self.writePDF)
        self.btn_writePdf.pack(side = tk.RIGHT, padx = 5)
        
        self.btn_bidinfo = ttk.Button(self.rf2, text = '參數列表', command = self.info_window)
        self.btn_bidinfo.pack(side = tk.RIGHT, padx = 5)
        
     #=================menu bar=============================
        self.menubar = tk.Menu(self.master)
        self.menu_file = tk.Menu(self.menubar, tearoff = 0)
        
        self.menu_file.add_command(label = '開啟資料夾', command = self.opendir)
        self.menu_file.add_separator()
        self.menu_file.add_command(label = '結束', command = self.master.destroy)
        self.menubar.add_cascade(label='檔案', menu = self.menu_file)
        
        self.menubar.add_command(label = '自動換行', command = self.wraptext)
        self.master.config(menu=self.menubar)
        
        self.menubar.add_command(label = '複製', command = self.copytoClip)
        self.master.config(menu=self.menubar)
        
        self.menubar.add_command(label = '關於', command = self.about)
        self.master.config(menu=self.menubar)
        
        
        
    def OptionMenu_SelectionEvent(self, event):
        print("value is", self.opt_var.get())
        x = self.folder2['text']
        tk.messagebox.showinfo('自動決包說明輔助工具', x)
        
    def update_opt(self):
        # Reset var and delete all old options
#         if filename == None:
#             curItem = self.tree.focus()
#             filename = self.tree.item(curItem)['text']
        filename = os.path.basename(self.pdf)
        self.path = os.path.dirname(self.pdf)
        self.opt_var.set('')
        self.opt['menu'].delete(0, 'end')
        new_choices = []
        new_choices.append(filename)
#         full path = os.path.join(self.tree.item(curItem)['value'][1], self.rename(filename))        
        if os.path.isfile(self.rename(self.pdf)) :
            new_choices.append(self.rename(filename))
        if new_choices[0].split('.')[0][-2:] != '-w' and len(new_choices) == 2 :
            new_choices.reverse()
        self.opt_var.set(new_choices[0])
        # Insert list of new options (tk._setit hooks them up to var)
        for choice in new_choices:
            self.opt['menu'].add_command(label=choice, command=tk._setit(self.opt_var, choice))

    def tree_refresh(self, path= None):
        if path == None:
            path = self.path
        print(path)
        self.tree.delete(*self.tree.get_children())
        self.folder1=self.tree.insert("", 0,text="ERP開啟", values=self.tree['columns'],\
                                      open = True, tags = ('folder',))
        self.folder2=self.tree.insert("", 1,text=path, values=self.tree['columns'],\
                                      open = True, tags = ('folder',))
        self.folder3=self.tree.insert("", 2,text=path+'(-w)', values=self.tree['columns'],\
                                      open = True, tags = ('folder',))       
        
        r = gf.walkpath()
        x = gf.append_pdftext(r)
        self.insert_tree(self.folder1, x)
        
        r = gf.getFileList(path, filetype = '.pdf')
        x = gf.append_pdftext(r)
        original = []
        edited = []
        for item in x:
            if item[0].split('.')[0][-2:] == '-w':
                print(item)
                edited.append(item)
            else:
                original.append(item)
        self.insert_tree(self.folder2, original)
        self.insert_tree(self.folder3, edited)
        
    def insert_tree(self, parent, li):
        i = 0
        for item in li:
            self.tree.insert(parent, "end",  text=item[0], iid = item[0], values=(item[2], item[1]))
            i +=1
    
    def make_submission(self):
        self.bid = rp.open(self.pdf)
        award_sn = self.bid.vndSn
        #刪除scrolledtxt內容
        self.txt.delete('1.0', tk.END)
        self.txt.insert(tk.INSERT, self.bid.submission_text(award_sn))
        self.statusbar.config(text='產生議價說明:'\
                              + self.bid.id + '-' + self.bid.comp+ '-' + self.bid.sn + '-' + self.bid.name)
    def get_editText(self):
        self.bid = rp.open(self.pdf)
        pos = {'x0': 50.000, 'top': 385.391, 'x1':818.000, 'bottom': 498.191}
        pdf_1 = pdfplumber.open(self.pdf)
        p0 = pdf_1.pages[0]
        a = rp.get_text(p0,pos)
        self.txt.delete('1.0', tk.END)
        self.txt.insert(tk.INSERT, a)
        self.statusbar.config(text='讀取'+ self.pdf + '的文字完成。')
    
    def selectItem(self, event):
        curItem = self.tree.focus()
#         print(self.tree.item(curItem))
        
        if self.tree.item(curItem)['values'][0] == 'None' or self.tree.item(curItem)['values'][0] == '案件名稱':
            self.statusbar.config(text='你選取的不是可載入的檔案。')
        else:
            filename = self.tree.item(curItem)['text']
            self.path = self.tree.item(curItem)['values'][1]
            self.pdf = os.path.join(self.path, filename)
            if self.tree.item(curItem)['text'].split('.')[0][-2:] == '-w' :
                self.get_editText()
            else:
                self.make_submission()

            self.update_opt()
    
    def copytoClip(self):
        clipboard.copy(self.txt.get("1.0", tk.END))
        self.statusbar.config(text = '議價說明已複製。')
        
    def wraptext(self):
        ans = simpledialog.askinteger('自動決包說明輔助工具', '自動換行的行數', initialvalue=38)

        if ans != None:
            text = self.txt.get("1.0", tk.END)
            self.txt.delete('1.0', tk.END)
            text = autowrap(text, ans)
            self.txt.insert(tk.INSERT, text)
            self.copytoClip()
        
    def info_window(self):
        win_info = tk.Toplevel(self.master)
        infoApp = info_app(win_info, self.bid)
    
    def remove(self):
        curItem = self.tree.focus()
        if self.tree.item(curItem)['values'][0] == 'None' or self.tree.item(curItem)['values'][0] == '案件名稱':
            self.statusbar.config(text='你選取的不是檔案。')
        else:
            full_path = os.path.join(self.tree.item(curItem)['values'][1],\
                                     self.tree.item(curItem)['text'])
            del self.bid
            os.remove(full_path)
            self.statusbar.config(text ='刪除'+ full_path +'完成。')
        
        self.opt_var.set('')
        self.opt['menu'].delete(0, 'end')
        
    def writePDF(self):
        userpath = self.tree.item(self.folder2)['text']
        if 'INetCache' in self.pdf:
            sep = '-'
            filename = sep.join((self.bid.id, self.bid.comp, self.bid.sn)) + '.pdf'
            full_path = os.path.join(userpath, filename)
            copyfile(self.pdf, full_path)
            self.statusbar.config(text = '複製ERP開啟檔案到' + full_path)
            self.pdf = full_path
        elif os.path.basename(self.pdf).split('.')[0][-2:] == '-w':
            if os.path.isfile(self.rename(self.pdf)) :
                self.pdf =self.rename(self.pdf)
            else:
                self.statusbar.config(text = '沒有原始檔案供寫入，請至ERP再下載一次。')
                return
            
        myText = self.txt.get("1.0", tk.END)
        self.statusbar.config(text='將文字寫入'+ self.rename(self.pdf) + '...')
        wp.write_pdf(self.pdf, myText, 12)
        self.statusbar.config(text='將文字寫入'+ self.rename(self.pdf) + '完成。')
        self.tree_refresh(userpath)
        self.update_opt()
    
    def printPDF(self):
        filename = self.opt_var.get()
        full_path = os.path.join(self.path, filename)
        self.statusbar.config(text='列印'+ full_path + '...')
        os.startfile(full_path,'print')
    
    def openPDF(self):
        filename = self.opt_var.get()
        full_path = os.path.join(self.path, filename)
        self.statusbar.config(text='打開'+ full_path + '...')
        os.startfile(full_path)
    
    def about(self):
        text = '''
            Something bigger then yourself to make you keep walking...
        '''
        tk.messagebox.showinfo('自動決包說明輔助工具', text)
    
    def opendir(self):
        self.path = tk.filedialog.askdirectory(initialdir = 'd:/')
        self.statusbar.config(text='讀取'+ self.path + '內的PDF檔...')
        self.tree_refresh(self.path)
        self.statusbar.config(text='讀取'+ self.path + '內的PDF檔完成')
    
    #寫入的pdf檔重新命名並製作新的路徑
    def rename(self, path):
        nf = os.path.basename(path).split('.')
        if nf[0][-2:] == '-w':
            f = path.replace('-w','')
        else:
            f = path.replace(nf[0], nf[0] + '-w')
        return f


def autowrap(text, width):  
    text = text.split('\n')
    #偵測項目符號字元，自動計算縮排空格數
    a = text[0][0:2]#取出第一行前2碼
    indent_char = ''
    for i in range(len(a.encode('gbk'))):
        indent_char = indent_char+' '
    
    newtext = ''
    for element in text:
        if len(element) < width:
            newtext = newtext + element + '\n'
        else:
            wrapt = textwrap.wrap(element, width)
            for x in range(len(wrapt)):
                if x == 0:
                    newtext = newtext + wrapt[x] + '\n'
                else:
                    newtext = newtext + textwrap.indent(wrapt[x], indent_char) + '\n'

    return newtext

def getFileList(dir ,type):
    files = os.listdir(dir)
    aList = []
    for file in files:
        full_path = os.path.join(dir, file)
        if os.path.isfile(full_path) and file[-3:] == type:
            aList.append(file)

    return aList

class info_app:
    def __init__(self, master, bid):
        self.master = master
        self.master.title('案件資訊')
        self.topframe = tk.LabelFrame(self.master, width= 700, text = '案件資訊')
        self.bottomframe = tk.LabelFrame(self.master, width = 700, text = '報價資訊')
        self.submit_btn = ttk.Button(self.master, text='關閉', command=self.submit)
        
        self.topframe.pack(padx = 10, pady = 5)
        self.bottomframe.pack(padx =10 , pady = 5)
        self.submit_btn.pack(pady = 5)        


        self.labelsBid = ['工號', '公司', '次別','工程名稱', '幣別', '性質', '預算']
        self.labelsQuo = ['廠商名稱','廠商編號','報價方式', '報價金額','報價稅率', '議價', '議價稅率']
        

        for i in range(len(self.labelsBid)):
            self.cur_label = ttk.Label(self.topframe, text = self.labelsBid[i])
            self.cur_label.grid(row=i, column=0, sticky=tk.NW, pady = 2)

        for i in range(len(self.labelsQuo)):
            self.cur_label = ttk.Label(self.bottomframe, text = self.labelsQuo[i])
            self.cur_label.grid(row = i , column = 0 , sticky = tk.W)

        self.name_var = tk.StringVar() 
        self.bid_info = {
            'id': tk.StringVar(),
            'comp': tk.StringVar(),
            'sn': tk.StringVar(),
            'name': tk.StringVar(),
            'currency': tk.StringVar(),
            'type': tk.StringVar(),
            'budget' : tk.StringVar()
        }

        self.quo_info = {
            'vndName' : tk.StringVar(),
            'vndID' : tk.StringVar(),
            'vndType': tk.StringVar(),
            'vndQuo': tk.StringVar(),
            'vndQuorate': tk.StringVar(),
            'vndFinalprice': tk.StringVar(),
            'vndFinalRate': tk.StringVar(),
        }


        counter = 0
        for i in self.bid_info:
            self.cur_entrybox = ttk.Entry(self.topframe, width=28 , textvariable=self.bid_info[i])
            self.cur_entrybox.grid(row=counter, column= 1, sticky = tk.E, padx = 5)
            counter += 1 

        counter = 0
        for i in self.quo_info:
            self.cur_entrybox = ttk.Entry(self.bottomframe, width=28 , textvariable=self.quo_info[i])
            self.cur_entrybox.grid(row=counter, column= 1, sticky = tk.E, padx = 5)
            counter += 1 
        
        self.initial_bid(bid)
        self.initial_quo(bid)

    def initial_bid(self, bid):
        self.bid_info['name'].set(bid.name)
        self.bid_info['id'].set(bid.id)
        self.bid_info['comp'].set(bid.comp)
        self.bid_info['sn'].set(bid.sn)
        self.bid_info['currency'].set(bid.currency)
        self.bid_info['type'].set(bid.type)
        self.bid_info['budget'].set(bid.budget)
    
    def initial_quo(self, bid):
        sn = 0
        self.quo_info['vndName'].set(bid.vnd[sn]['name'])
        self.quo_info['vndID'].set(bid.vnd[sn]['id'])
        self.quo_info['vndType'].set(bid.vnd[sn]['type'])
        self.quo_info['vndQuo'].set(bid.vnd[sn]['quo'])
        self.quo_info['vndQuorate'].set(bid.vnd[sn]['quoRate'])
        self.quo_info['vndFinalprice'].set(bid.vnd[sn]['finalPrice'])
        self.quo_info['vndFinalRate'].set(bid.vnd[sn]['finalRate'])
    
    def submit(self):
        self.master.destroy()

        

# mycase = rp.open('d:/T1.pdf')
# print(mycase.submission_text())
root = tk.Tk()
app = App(root)
root.mainloop()




d:/
['58495M62-4LD-01-w.pdf', 'd:/', '寧波PTA廠設備管線檢測019']
['T1-w.pdf', 'd:/', '一二廠衛生間改善']
['T2-w.pdf', 'd:/', '寧波工事處辦公樓外墻面改善工程']
['T5-w.pdf', 'd:/', '厂区篮球场及倒班楼楼梯墙裙修复']


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\n000139350\miniconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-2-7dd54fbd7d11>", line 213, in selectItem
    self.make_submission()
  File "<ipython-input-2-7dd54fbd7d11>", line 187, in make_submission
    self.txt.insert(tk.INSERT, self.bid.submission_text(award_sn))
  File "<string>", line 189, in submission_text
UnboundLocalError: local variable 'part3' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\users\n000139350\miniconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-2-7dd54fbd7d11>", line 213, in selectItem
    self.make_submission()
  File "<ipython-input-2-7dd54fbd7d11>", line 187, in make_submission
    self.txt.insert(tk.INSERT, self.bid.submission_text(award_sn))
  File "<string>", line 189, in submission_text
UnboundLocalE